# Blockchain Mining Simulation with Verbose Logging

In this notebook, we simulate blockchain mining and display detailed internal workings during the mining process.

**Steps Covered:**
1. Define the Block structure.
2. Implement a mining function with verbose logging.
3. Create a Blockchain class.
4. Simulate mining multiple blocks and display the chain.


In [ ]:
# Importing necessary libraries
import hashlib  # For hashing the block data using SHA-256
import time     # To timestamp each block

## Part 1: Block Structure

In this cell, we define a `Block` class. Each block in our blockchain will contain:
- **index:** Position of the block in the blockchain.
- **timestamp:** Time when the block was created.
- **data:** A list of transactions or data for the block.
- **previous_hash:** The hash of the previous block in the chain.
- **nonce:** A number that miners adjust during mining until a valid hash is found.
- **hash:** The resulting SHA-256 hash of the block's contents.

Let's define the `Block` class.


In [ ]:
# Define the Block class with detailed inline comments
class Block:
    def __init__(self, index, previous_hash, data):
        # Set the block's position in the chain
        self.index = index
        # Record the time when the block is created
        self.timestamp = time.time()
        # Store the transactions or data for the block
        self.data = data
        # Reference the previous block's hash for linking the blocks
        self.previous_hash = previous_hash
        # Initialize the nonce (number used once) which will be changed during mining
        self.nonce = 0
        # Compute the initial hash of the block
        self.hash = self.compute_hash()

    def compute_hash(self):
        """
        Compute the SHA-256 hash of the block's contents.
        Combines index, timestamp, data, previous hash, and nonce.
        """
        block_content = f"{self.index}{self.timestamp}{self.data}{self.previous_hash}{self.nonce}"
        return hashlib.sha256(block_content.encode()).hexdigest()


## Part 2: Mining Function with Verbose Logging

The `mine_block` function performs proof-of-work by adjusting the nonce until the block's hash starts with a target prefix (a set number of zeros). Here, we print detailed information for every iteration to show how the nonce and hash change.



In [ ]:
def mine_block(block, difficulty=4, verbose=True):
    """
    Mines a block by finding a nonce such that the block's hash starts with 'difficulty' zeros.
    Prints detailed debug information on each iteration if verbose is True.
    """
    prefix = '0' * difficulty
    print(f"\nStarting mining for block {block.index} with target prefix: {prefix}")
    iteration = 0
    while not block.hash.startswith(prefix):
        block.nonce += 1
        block.hash = block.compute_hash()
        iteration += 1
        if verbose:
            print(f"Iteration {iteration}: nonce = {block.nonce}, hash = {block.hash}")
    print(f"Block {block.index} mined! Final nonce: {block.nonce}, hash: {block.hash}\n")
    return block.nonce, block.hash


## Part 3: Blockchain Implementation

Here we implement the `Blockchain` class that will manage our chain of blocks. This class includes:

- **create_genesis_block():** Creates the first block (genesis block) in the blockchain.
- **add_block(data):** Adds a new block containing the provided data by mining it.
- **is_chain_valid():** Checks the validity of the blockchain by verifying each block's hash and linkage.
- **display_chain():** Prints out the details of each block in the blockchain.

Let's define the `Blockchain` class.


In [ ]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.difficulty = 4  # Difficulty level for mining (number of leading zeros required)
        self.create_genesis_block()

    def create_genesis_block(self):
        # Create the first block with default previous hash "0"
        genesis_block = Block(0, "0", "Genesis Block")
        mine_block(genesis_block, self.difficulty, verbose=True)
        self.chain.append(genesis_block)

    def add_block(self, data):
        # Create a new block using the hash of the latest block in the chain
        last_block = self.chain[-1]
        new_block = Block(len(self.chain), last_block.hash, data)
        mine_block(new_block, self.difficulty, verbose=True)
        self.chain.append(new_block)

    def is_chain_valid(self):
        # Verify that each   is correct and that the chain is properly linked
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]
            if current.hash != current.compute_hash():
                return False
            if current.previous_hash != previous.hash:
                return False
        return True

    def display_chain(self):
        # Print detailed information for each block in the blockchain
        for block in self.chain:
            print(f"Index: {block.index}")
            print(f"Timestamp: {block.timestamp}")
            print(f"Data: {block.data}")
            print(f"Previous Hash: {block.previous_hash}")
            print(f"Nonce: {block.nonce}")
            print(f"Hash: {block.hash}")
            print("-" * 50)


## Part 4: Mining Simulation

In this cell, we will:
- Initialize our blockchain.
- Add several blocks with sample transaction data.
- Display the blockchain after adding each block.
- Validate the integrity of the entire blockchain using the `is_chain_valid()` method.


In [ ]:
# Initialize the blockchain (automatically creates the genesis block)
blockchain = Blockchain()

# Add new blocks with sample transaction data
blockchain.add_block("Transaction A to B: $50")
blockchain.add_block("Transaction C to D: $20")
blockchain.add_block("Transaction E to F: $100")

# Display the entire blockchain with detailed block information
blockchain.display_chain()

# Validate the blockchain integrity and print the result
print("Is blockchain valid?", blockchain.is_chain_valid())

OR

In [ ]:
# Display the hash of all blocks in the blockchain
print("Hashes of all blocks in the blockchain:")
for block in blockchain.chain:
    print(f"Block {block.index} hash: {block.hash}")

In [ ]:
def verify_chain_details(blockchain):
    print("Starting blockchain verification...\n")
    is_valid = True
    # Start from block 1 because the genesis block doesn't have a previous block to verify
    for i in range(1, len(blockchain.chain)):
        current = blockchain.chain[i]
        previous = blockchain.chain[i - 1]
        print(f"Verifying block {current.index}:")
        
        # Step 1: Recompute the hash for the current block
        computed_hash = current.compute_hash()
        print(f"  Stored hash:    {current.hash}")
        print(f"  Computed hash:  {computed_hash}")
        
        if current.hash != computed_hash:
            print("  [Error] Block hash does not match computed hash!")
            is_valid = False
        else:
            print("  [OK] Block hash is valid.")
        
        # Step 2: Verify the linkage by checking the previous hash
        print(f"  Previous hash in current block: {current.previous_hash}")
        print(f"  Actual previous block's hash:    {previous.hash}")
        if current.previous_hash != previous.hash:
            print("  [Error] Previous hash does not match!")
            is_valid = False
        else:
            print("  [OK] Previous hash is valid.")
        
        print("-" * 50)
    
    if is_valid:
        print("Blockchain verification complete: The blockchain is VALID!")
    else:
        print("Blockchain verification complete: The blockchain is INVALID!")
    
    return is_valid

# Now call the detailed verification function on the blockchain
verify_chain_details(blockchain)



## Part 5: Testing and Reflection

We have:
- Added multiple blocks to the blockchain.
- Displayed the complete blockchain with each block's details.
- Validated the blockchain to ensure integrity.

**Key Takeaways:**
- **Mining:** Adjusting the nonce until a block's hash meets the proof-of-work requirement.
- **Block Linkage:** Each block references the previous block's hash, forming an immutable chain.
- **Blockchain Integrity:** The `is_chain_valid()` method ensures that no block has been tampered with.

Feel free to experiment further by:
- Changing the mining difficulty.
- Adding more blocks.
- Modifying the transaction data.
